In [1]:
from mxnet.gluon import nn

In [3]:
net = nn.Sequential()
drop_prob1 = 0.2
drop_prob2 = 0.5

with net.name_scope():
    net.add(nn.Flatten())
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dropout(drop_prob1))
    net.add(nn.Dense(256, activation='relu'))
    net.add(nn.Dropout(drop_prob2))
    net.add(nn.Dense(10))
net.initialize()

In [4]:
import sys
sys.path.append('..')
import utils
from mxnet import nd
from mxnet import autograd
from mxnet import gluon

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.5})

for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.845618, Train acc 0.686699, Test acc 0.788361
Epoch 1. Loss: 0.524934, Train acc 0.807993, Test acc 0.834535
Epoch 2. Loss: 0.456228, Train acc 0.834034, Test acc 0.855869
Epoch 3. Loss: 0.418010, Train acc 0.847923, Test acc 0.863582
Epoch 4. Loss: 0.399864, Train acc 0.852881, Test acc 0.869591
